In [1]:
from langchain_community.tools import WikipediaQueryRun, ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

import subprocess
# subprocess.run(["ollama",'pull',"gemma3:270m"],check=True)
subprocess.run(["ollama",'pull',"embeddinggemma:300m"],check=True)

import os
from dotenv import load_dotenv

os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

load_dotenv()

c:\Users\rahul\anaconda3\envs\genai\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
USER_AGENT environment variable not set, consider setting it to identify your requests.


True

In [2]:
wiki_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=wiki_wrapper)
wiki.name

'wikipedia'

In [3]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [4]:
tools = [wiki, arxiv]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\rahul\\anaconda3\\envs\\genai\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200))]

In [5]:
# Create Custom Tool
docs = WebBaseLoader("https://docs.langchain.com/langsmith").load()
splits = RecursiveCharacterTextSplitter(chunk_size = 400, chunk_overlap = 100).split_documents(documents=docs)
embeddings = OllamaEmbeddings(model="embeddinggemma:300m")
faiss = FAISS.from_documents(documents=docs, embedding=embeddings)
retriever = faiss.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000125AE512F30>, search_kwargs={})

In [6]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever=retriever, name='langsmith-search',description='Get information about langsmith')
retriever_tool.name

'langsmith-search'

In [7]:
# # Initilaise the model

# llm = OllamaLLM(model='gemma3:270m')
# parser = StrOutputParser()

# chain = llm | parser
# chain.invoke("Explain in simple terms why do we use feedforward networks in transformers")

In [8]:
from langchain import hub
# prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt.messages

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# This mirrors the "openai-functions-agent" style prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the provided tools to answer the user’s questions."),
    MessagesPlaceholder(variable_name="chat_history"),  # keeps track of history
    ("user", "{input}"),                                # new user input
    MessagesPlaceholder(variable_name="agent_scratchpad"),  # for reasoning/tool use
])

prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful AI assistant. Use the provided tools to answer the user’s questions.'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history'),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [9]:
## Agents

from langchain.agents import create_openai_tools_agent
llm = ChatGroq(model='compound', streaming=True)
agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChun

In [10]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools= tools, verbose=True)
agent_executor.invoke({"input":"tell me about langsmith",
                       "chat_history": []})



> Entering new AgentExecutor chain...


AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
